In [ ]:
import numpy as np

In [ ]:
!pip install torch

In [ ]:
import pandas as pd

In [ ]:
path = '/dgxa_home/se22uari173/train.csv'

In [ ]:
df = pd.read_csv(path)

In [868]:
df

,id,dialogue,summary,topic
0,train_0,"#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. ...","Mr. Smith's getting a check-up, and Doctor Haw...",get a check-up
1,train_1,"#Person1#: Hello Mrs. Parker, how have you bee...",Mrs Parker takes Ricky for his vaccines. Dr. P...,vaccines
2,train_2,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,find keys
3,train_3,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,have a girlfriend
4,train_4,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,dance
...,...,...,...,...
12455,train_12455,#Person1#: Excuse me. You are Mr. Green from M...,Tan Ling picks Mr. Green up who is easily reco...,pick up someone
12456,train_12456,#Person1#: Mister Ewing said we should show up...,#Person1# and #Person2# plan to take the under...,conference center
12457,train_12457,#Person1#: How can I help you today?\n#Person2...,#Person2# rents a small car for 5 days with th...,rent a car
12458,train_12458,#Person1#: You look a bit unhappy today. What'...,#Person2#'s mom lost her job. #Person2# hopes ...,job losing


In [ ]:
!pip install spacy

In [ ]:
import spacy

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
import torch

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def preprocess_text(text):
    doc = nlp(text)
    cleaned_tokens = [
        token.text for token in doc if not token.is_punct and not token.is_space
    ]
    return " ".join(cleaned_tokens)

In [ ]:
import numpy as np
percent = 97
print(np.percentile([len(x) for x in df['dialogue']],percent))

In [ ]:
print(np.percentile([len(x) for x in df['summary']],percent))

In [ ]:
max_input_seq_length = 1584
max_output_seq_length = 290

In [ ]:
dialogue_list = list(df['dialogue'])

In [ ]:
summary_list = list(df['summary'])

In [ ]:
for i in range(len(dialogue_list)):
    dialogue_list[i] = preprocess_text(dialogue_list[i])
for i in range(len(summary_list)):
    summary_list[i] = preprocess_text(summary_list[i])

In [ ]:
maxi = 0
for i in dialogue_list:
    if(len(i)>maxi):
        maxi = len(i)

In [ ]:
maxi

In [ ]:
def is_validlength(paragraph,max_seq_length):
    return(len(paragraph)<max_input_seq_length)
valid_textsummaries_idx = []
for i in range(len(df['summary'])):
    dialogue,summary = dialogue_list[i],summary_list[i]
    if(is_validlength(dialogue,max_input_seq_length) and is_validlength(summary,max_output_seq_length)):
        valid_textsummaries_idx.append(i)

In [ ]:
len(valid_textsummaries_idx)

12085

In [ ]:
dialogue_list_new = []
summary_list_new = []
for i in valid_textsummaries_idx:
    dialogue_list_new.append(dialogue_list[i])
    summary_list_new.append(summary_list[i])

In [ ]:
from torch.utils.data import Dataset,DataLoader

In [ ]:
class TextDataset(Dataset):
    def __init__(self,dialogues,summaries):
        self.dialogues = dialogues
        self.summaries = summaries
    def __len__(self):
        return len(self.summaries)
    def __getitem__(self,j):
        return self.dialogues[j],self.summaries[j]

In [ ]:
data = TextDataset(dialogue_list_new,summary_list_new)

In [ ]:
data[0]

("#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor.",
 "Mr. Smith's getting a check-up, and Doctor Hawkins 

In [ ]:
data = TextDataset(dialogue_list_new, summary_list_new)
batch_size = 1
train_dataset = DataLoader(data, batch_size=batch_size, shuffle=True)

In [ ]:
c = 1
for batch in train_dataset:
    dialogues,summaries = batch
    print(summaries)
    if(c==1):
        break

In [894]:
df['dialogue'][0]

"#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor."

In [896]:
len(data)

12085

In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

In [ ]:
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")

In [ ]:
len(train_dataset)

In [ ]:
max = 0
for i in dialogue_list_new:
    if(max<len(tokenizer(i)['input_ids'])):
        max = len(tokenizer(i)['input_ids'])
print(max)

In [ ]:
max = 0
for i in summary_list_new:
    if(max<len(tokenizer(i)['input_ids'])):
        max = len(tokenizer(i)['input_ids'])
print(max)

In [ ]:
len(batch)

In [ ]:
input_ids = []
output_ids = []
max_input_seq_length = 450
max_output_seq_length = 450
for batch in train_dataset:
    if(len(batch)==2):
        dialogues, summaries = batch 
        tokenized_dialogues = tokenizer(list(dialogues),max_length=max_input_seq_length,padding='max_length',return_tensors='pt',add_special_tokens=False)
        tokenized_summaries = tokenizer(list(summaries),max_length=max_output_seq_length,padding='max_length',return_tensors='pt',add_special_tokens=True)
        input_ids.append(tokenized_dialogues['input_ids'])
        output_ids.append(tokenized_summaries['input_ids'])

In [912]:
len(output_ids)

6043

In [ ]:
tensor_tokenized_inputs = torch.cat(input_ids,dim=0)

In [ ]:
tensor_tokenized_outputs = torch.cat(output_ids,dim=0)

In [916]:
for i in range(10):
    print(input_ids[i])

tensor([[ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0]])
tensor([[ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0]])
tensor([[ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0]])
tensor([[ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0]])
tensor([[ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0]])
tensor([[ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0]])
tensor([[ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0]])
tensor([[ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0]])
tensor([[ 1713,   345, 13515,  ...,     0,     0,     0],
      

In [918]:
tensor_tokenized_outputs = torch.cat(output_ids,dim=0)

In [920]:
tensor_tokenized_inputs

tensor([[ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0],
        ...,
        [ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0],
        [ 1713,   345, 13515,  ...,     0,     0,     0]])

In [922]:
len(tensor_tokenized_inputs)

12085

In [ ]:
import math
from torch import nn
import torch.nn.functional as func

In [ ]:
class Multiheadattention(nn.Module):
    def __init__(self,weights_dim,n_heads):
        super().__init__()
        self.weights_dim = weights_dim
        self.n_heads = n_heads
        self.head_dimensions = weights_dim//n_heads
        self.qkv_matrices = nn.Linear(weights_dim,3*weights_dim)
        self.linear_layer = nn.Linear(weights_dim,weights_dim)
    def forward(self,x,mask):
        batch,maxseqlength,weight_dim = x.size()
        qkv = self.qkv_matrices(x)
        qkv = qkv.reshape(batch,maxseqlength,self.n_heads,3*self.head_dimensions)
        qkv = qkv.permute(0,2,1,3)
        q,k,v = qkv.chunk(3,dim=-1)
        k_transpose = k.transpose(-1,-2)
        product = torch.matmul(q,k_transpose)
        product = product/math.sqrt(q.size()[-1])
        if mask is not None:
            product = product+mask
        attention_scores = func.softmax(product,dim=-1)
        attentionintoV = torch.matmul(attention_scores,v)
        n = (self.n_heads*self.head_dimensions)
        output = self.linear_layer(attentionintoV.reshape(batch,maxseqlength,n))
        return output

In [ ]:
class LayerNormalization(torch.nn.Module):
    def __init__(self,eps):
        super(LayerNormalization, self).__init__()
        self.eps = eps 
    def forward(self,inp):
        mean = inp.mean(dim=-1, keepdim=True) 
        var = ((inp - mean) ** 2).mean(dim=-1, keepdim=True)
        std = torch.sqrt(var + self.eps)
        y = (inp - mean) / std
        return y

In [ ]:
class feedfowardlayer(nn.Module):
    def __init__(self,weights_dims,hidden_dim,drop_prob):
        super(feedfowardlayer, self).__init__()
        self.weights_dims = weights_dims
        self.hidden_dims = hidden_dim
        self.linearlayer1 = nn.Linear(self.weights_dims,self.hidden_dims)
        self.relu = nn.ReLU()
        self.linearlayer2 = nn.Linear(self.hidden_dims,self.weights_dims)
        self.drop_probability = drop_prob
        self.drop_neurons = nn.Dropout(p=self.drop_probability)
    def forward(self,x):
        out = self.linearlayer1(x)
        out = self.relu(out)
        out = self.drop_neurons(out)
        out = self.linearlayer2(out)
        return out

In [ ]:
class Subencoder(nn.Module):  # its one of the encoder layers in pipeline of encoder
    def __init__(self,weight_dim,ff_hidden,n_heads,drop_probability):
        super(Subencoder,self).__init__()
        self.attention = Multiheadattention(weight_dim,n_heads)
        self.norm = LayerNormalization(1e-6)
        self.dropout1 = nn.Dropout(p = drop_probability)
        self.ffd = feedfowardlayer(weight_dim,ff_hidden,drop_probability)
        self.norm2 = LayerNormalization(1e-6)
        self.dropout2 = nn.Dropout(p = drop_probability)
    def forward(self,x,mask):
        residual = x
        x = self.attention(x,mask)
        x = self.dropout1(x)
        x = self.norm(x+residual)
        residual = x
        x = self.ffd(x)
        x = self.dropout2(x)
        x = self.norm2(x+residual)
        return x

In [ ]:
class Sequentialencoder(nn.Sequential): #it makes pipeline for multiple encoders
    def foward(self,x,mask):
        self.x = x
        self.mask = mask
        for module in self._modules.values():
            x = module(self.x,self.mask)
        return y

In [ ]:
class Encoder(nn.Module):
    def __init__(self,weight_dim,ff_hidden,n_heads,layers,drop_probability):
        super(Encoder, self).__init__()
        encoded_layers = []
        for k in range(layers): 
            l = Subencoder(weight_dim, ff_hidden,n_heads,drop_probability)
            encoded_layers.append(l)
        self.layers = nn.ModuleList(encoded_layers)
    def forward(self, x, mask=None):
        for layer in self.layers:
            x = layer(x, mask)
        return x

In [ ]:
class Multiheadcrossattention(nn.Module):
    def __init__(self,weights_dim,n_heads):
        super().__init__()
        self.weights_dim = weights_dim
        self.n_heads = n_heads
        self.head_dimensions = weights_dim//n_heads
        self.kv_matrices = nn.Linear(weights_dim,2*weights_dim)
        self.q_matrix = nn.Linear(weights_dim,weights_dim)
        self.linear_layer = nn.Linear(weights_dim,weights_dim)
    def forward(self,y,x,mask):
        batches,maxseqlength,weight_dim = x.size()
        kv = self.kv_matrices(x)
        q = self.q_matrix(y)
        kv = kv.reshape(batches,maxseqlength,self.n_heads,2*self.head_dimensions)
        kv = kv.permute(0,2,1,3)
        k,v = kv.chunk(2,dim=-1)
        q = q.reshape(batches,maxseqlength,self.n_heads,self.head_dimensions)
        q = q.permute(0,2,1,3)
        k_transpose = k.transpose(-1,-2)
        product = torch.matmul(q,k_transpose)
        product = product/math.sqrt(q.size()[-1])
        if mask is not None:
            product = product+mask
        attention_scores = func.softmax(product,dim=-1)
        attentionintoV = torch.matmul(attention_scores,v)
        n = (self.n_heads*self.head_dimensions)
        output = self.linear_layer(attentionintoV.reshape(batches,maxseqlength,n))
        return output

In [ ]:
class Maskedattention(nn.Module):
    def __init__(self,weights_dim,n_heads):
        super().__init__()
        self.weights_dim = weights_dim
        self.n_heads = n_heads
        self.head_dimensions = weights_dim//n_heads
        self.qkv_matrices = nn.Linear(weights_dim,3*weights_dim)
        self.linear_layer = nn.Linear(weights_dim,weights_dim)
    def forward(self,x,mask,padmask):
        batch,maxseqlength,weight_dim = x.size()
        qkv = self.qkv_matrices(x)
        qkv = qkv.reshape(batch,maxseqlength,self.n_heads,3*self.head_dimensions)
        qkv = qkv.permute(0,2,1,3)
        q,k,v = qkv.chunk(3,dim=-1)
        k_transpose = k.transpose(-1,-2)
        product = torch.matmul(q,k_transpose)
        product = product/math.sqrt(q.size()[-1])
        if mask is not None:
            product = product+mask+padmask
        attention_scores = func.softmax(product,dim=-1)
        attentionintoV = torch.matmul(attention_scores,v)
        n = (self.n_heads*self.head_dimensions)
        output = self.linear_layer(attentionintoV.reshape(batch,maxseqlength,n))
        return output

In [ ]:
class Subdecoder(nn.Module):
    def __init__(self,weight_dim,ff_hidden,n_heads,drop_probability):
        super(Subdecoder,self).__init__()
        self.maskedattention = Maskedattention(weight_dim,n_heads)
        self.norm = LayerNormalization(1e-6)
        self.dropout1 = nn.Dropout(p = drop_probability)
        self.Multicrossattention = Multiheadcrossattention(weight_dim,n_heads)
        self.norm2 = LayerNormalization(1e-6)
        self.dropout2 = nn.Dropout(p = drop_probability)
        self.ffd = feedfowardlayer(weight_dim,ff_hidden,drop_probability)
        self.norm3 = LayerNormalization(1e-6)
        self.dropout3 = nn.Dropout(p = drop_probability)
    def forward(self,x,y,mask,padmask,crossmask):
        residual = y
        y = self.maskedattention(y,mask,padmask)
        y = self.dropout1(y)
        y = self.norm(y+residual)
        residual = y
        y = self.Multicrossattention(y,x,crossmask)
        y = self.dropout2(y)
        y = self.norm2(y+residual)
        residual = y
        y = self.ffd(y)
        y = self.dropout3(y)
        y = self.norm3(y+residual)
        return y

In [ ]:
class Sequentialdecoder(nn.Sequential):
    def forward(self,x,y,mask,padmask,crossattentpadmask):
        for module in self._modules.values():
            y = module(x,y,mask,padmask,crossattentpadmask)
        return y

In [ ]:
class Decoder(nn.Module):
    def __init__(self,weight_dim,ff_hidden,n_heads,layers,drop_probability):
        super().__init__()
        decoded_layers = []
        for k in range(layers):
            l = Subdecoder(weight_dim,ff_hidden,n_heads,drop_probability)
            decoded_layers.append(l)
        self.layers = Sequentialdecoder(*[Subdecoder(weight_dim,ff_hidden,n_heads,drop_probability)])
    def forward(self,x,y,mask,padmask,crossattentpadmask):
        resdecoder = self.layers(x,y,mask,padmask,crossattentpadmask)
        return resdecoder

In [ ]:
vocab_size = tokenizer.vocab_size

In [ ]:
vocab_size

32000

In [ ]:
class PositionalEncodingWithFeatureTransformation(nn.Module):
    def __init__(self, input_dim, output_dim, max_sequence_length):
        super(PositionalEncodingWithFeatureTransformation, self).__init__()
        self.max_sequence_length = max_sequence_length
        self.output_dim = output_dim
        self.linear = nn.Linear(input_dim, output_dim)
        self.positional_embeddings = nn.Parameter(torch.zeros(max_sequence_length, output_dim), requires_grad=True)
        self.create_positional_encodings()
    def create_positional_encodings(self):
        even_i = torch.arange(0, self.output_dim, 2).float()
        denominator = torch.pow(10000, even_i / self.output_dim)
        position = torch.arange(self.max_sequence_length).reshape(self.max_sequence_length, 1)
        even_PE = torch.sin(position / denominator)
        odd_PE = torch.cos(position / denominator)
        stacked = torch.stack([even_PE, odd_PE], dim=2)
        self.positional_embeddings.data = torch.flatten(stacked, start_dim=1, end_dim=2)
    def forward(self, x):
        x = self.linear(x)  
        positional_encoded = self.positional_embeddings[:x.size(1), :].unsqueeze(0) 
        return x + positional_encoded

In [ ]:
embedding_matrix_inputs = nn.Linear(512,512)
embedding_matrix_outputs = nn.Linear(512,512)
positional_embeddings = PositionalEncodingWithFeatureTransformation(input_dim=512,output_dim=512,max_sequence_length=450)
optimizer = torch.optim.Adam(list(transformer.parameters()) + list(embedding_matrix_inputs.parameters()) + list(embedding_matrix_outputs.parameters()) + list(positional_embeddings.parameters()),lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
word2vec = Word2VecEmbedding(32000,512)
for i in range(30):
    weights_dim = 512
    ff_hidden = 2048
    n_heads = 8
    drop_probability = 0.15
    num_layers = 3
    batch_size = 1
    for i in range(len(train_dataset)):
        start_idx = i * batch_size
        end_idx = start_idx + batch_size
        current_batch_inputs = tensor_tokenized_inputs[start_idx:end_idx]  
        current_batch_outputs = tensor_tokenized_outputs[start_idx:end_idx]
    with torch.no_grad(): 
        X = word2vec(current_batch_inputs) 
    with torch.no_grad():
        Y = word2vec(current_batch_outputs)
    embedded_X = embedding_matrix_inputs(X)
    embedded_Y = embedding_matrix_outputs(Y)
    positional_embeddingsx = positional_embeddings(X) 
    final_X = embedded_X + positional_embeddingsx
    positional_embeddingsy = positional_embeddings(Y) 
    final_Y = embedded_Y + positional_embeddingsy
    src_mask = create_padding_mask(current_batch_inputs,1,450,8)
    tgt_mask = create_padding_mask(current_batch_outputs,1,450,8)
    lkahd_mask = create_lookahead_mask(450,8)
    crossattention_mask = create_padding_mask_2(current_batch_outputs,450,8)
    logits = transformer(final_X,final_Y,src_mask,tgt_mask,lkahd_mask,crossattention_mask)
    probabilities = func.softmax(logits,dim=-1)
    predicted_tokens = torch.argmax(probabilities,dim=-1)
    predicted_text = tokenizer.decode(predicted_tokens[0].tolist())
    loss = criterion(logits.view(-1, logits.size(-1)), current_batch_outputs.view(-1))
    print(loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()